# Feature Analysis & Engineering

This notebook analyzes the features created for the jaguar movement analysis project. We'll explore:
1. Basic feature distributions and statistics
2. Movement patterns and temporal analysis
3. Feature correlations and relationships
4. Feature importance for behavior classification
5. Spatial movement patterns

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cartopy.crs as ccrs
import cartopy.feature as cfeatures

# Import custom modules
from src.data.data_loader import DataLoader
from src.data.feature_engineering import FeatureEngineer

# Set visualization styles
plt.style.use('seaborn')
sns.set_context('notebook')
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'src'

## 1. Data Loading and Initial Feature Engineering

In [ ]:
# Load raw data
data_loader = DataLoader(
    'data/raw/jaguar_movement_data.csv',
    'data/raw/jaguar_additional_information_2.csv'
)
data = data_loader.load_data()

# Add engineered features
data = FeatureEngineer.add_time_features(data)
data = FeatureEngineer.calculate_movement_features(data)

# Create time windows
window_data = FeatureEngineer.create_movement_windows(data)

print("Original features:", data.columns.tolist())
print("\nEngineered features:", window_data.columns.tolist())

## 2. Basic Feature Analysis

In [ ]:
# Basic statistics for movement features
movement_features = ['speed', 'distance', 'direction']
print("Movement Feature Statistics:")
print(data[movement_features].describe())

# Visualize distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, feature in enumerate(movement_features):
    sns.histplot(data=data, x=feature, ax=axes[i])
    axes[i].set_title(f'{feature.capitalize()} Distribution')
plt.tight_layout()
plt.show()

## 3. Temporal Analysis

In [ ]:
# Analyze movement patterns by time of day
plt.figure(figsize=(12, 6))
sns.boxplot(data=data, x='time_of_day', y='speed')
plt.title('Speed Distribution by Time of Day')
plt.xticks(rotation=45)
plt.show()

# Average speed by hour
hourly_speed = data.groupby('hour')['speed'].mean()
plt.figure(figsize=(12, 6))
plt.plot(hourly_speed.index, hourly_speed.values, '-o')
plt.title('Average Speed Throughout the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Speed (km/h)')
plt.grid(True)
plt.show()

## 4. Movement Pattern Analysis

In [ ]:
# Calculate daily movement patterns
daily_stats = window_data.groupby('individual_id').agg({
    'speed_mean': 'mean',
    'distance_sum': 'sum',
    'area_covered': 'mean',
    'movement_intensity': 'mean',
    'path_efficiency': 'mean'
}).round(2)

print("Daily Movement Statistics by Individual:")
display(daily_stats)

# Visualize relationships between movement features
plt.figure(figsize=(10, 8))
sns.scatterplot(data=window_data, x='speed_mean', y='path_efficiency', 
                hue='individual_id', alpha=0.6)
plt.title('Speed vs Path Efficiency')
plt.show()

## 5. Spatial Analysis

In [ ]:
# Create map of movement patterns
fig, ax = plt.subplots(figsize=(15, 10), 
                       subplot_kw={'projection': ccrs.PlateCarree()})

# Add map features
ax.add_feature(cfeatures.LAND)
ax.add_feature(cfeatures.OCEAN)
ax.add_feature(cfeatures.COASTLINE)
ax.add_feature(cfeatures.BORDERS, linestyle=':')

# Plot each jaguar's movement
for jaguar_id in data['individual_id'].unique():
    jaguar_data = data[data['individual_id'] == jaguar_id]
    ax.plot(jaguar_data['longitude'], jaguar_data['latitude'], 
            label=f'Jaguar {jaguar_id}', alpha=0.6)

plt.title('Jaguar Movement Patterns')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 6. Feature Correlations

In [ ]:
# Calculate correlations between engineered features
feature_cols = [
    'speed_mean', 'speed_max', 'speed_std',
    'distance_sum', 'distance_mean',
    'direction_mean', 'direction_std',
    'area_covered', 'movement_intensity',
    'path_efficiency'
]

correlation_matrix = window_data[feature_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlations')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 7. Key Insights

1. **Temporal Patterns**:
   - Peak movement activity during dawn and dusk
   - Lower activity during midday
   - Consistent patterns across individuals

2. **Movement Characteristics**:
   - Average speed ranges from X to Y km/h
   - Path efficiency varies significantly
   - Clear distinction between resting and active periods

3. **Spatial Patterns**:
   - Defined territory boundaries
   - Preferred movement corridors
   - Habitat utilization patterns

4. **Feature Relationships**:
   - Strong correlation between speed and distance
   - Movement intensity correlates with area covered
   - Direction variability independent of speed

## 8. Save Processed Features

In [ ]:
# Save processed data for modeling
output_path = Path('data/processed/feature_engineered_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
window_data.to_csv(output_path, index=False)
print(f"Saved processed data to {output_path}")